In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm

In [2]:
# random number generator
import numpy as np

def LinearCongruentialIterator(seed=1):
    k = 2**31-1
    a, c = 39373, 0
    x = seed
    while True:
        x = (x * a + c) % k
        yield x / k

gen = LinearCongruentialIterator()

def LinearCongruentialGenerator(n):
    result = np.zeros(n)
    for i in range(n):
        result[i] = next(gen)
    return result

def BoxMuller(N):
    """
    Box-Muller method
    """
    cnt = 0
    res = []
    i = 0
    while len(res) < N:
        X = 2
        while X > 1:
            u1, u2 = LinearCongruentialGenerator(2)
            cnt += 2
            u1 = 2 * u1 - 1
            u2 = 2 * u2 - 1
            X = u1**2 + u2**2
        Y = np.sqrt(-2 * np.log(X) / X)
        Z1 = u1 * Y
        Z2 = u2 * Y
        res.append(Z1)
        res.append(Z2)

    return np.array(res)

In [3]:
# question 1.1
# parameters
a0 = 2.50662823884    
b0 = -8.47351093090
a1 = -18.61500062529
b1 = 23.08336743743
a2 = 41.39119773534
b2 = -21.06224101826
a3 = -25.44106049637
b3 = 3.13082909833
c0 = 0.3374754822726147
c5 = 0.0003951896511919
c1 = 0.9761690190917186
c6 = 0.0000321767881768
c2 = 0.1607979714918209
c7 = 0.0000002888167364
c3 = 0.0276438810333863
c8 = 0.0000003960315187
c4 = 0.0038405729373609

# Beasley–Springer–Moro algorithm function
def bsm(u: float):
    """
    intput: u is between 0 and 1
    output: x, approximation to \phi^-1(u)
    """
    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0: r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4+ r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0: 
            x = -x
    return x
        

In [4]:
# question 1.2
def black_scholes(S, K, T, r, q, sigma, option_type):

    d1 = (np.log(S / K) + (r - q + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        option_price = S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        delta = np.exp(-q * T) * norm.cdf(d1)
    elif option_type == 'put':
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * np.exp(-q * T) * norm.cdf(-d1)
        delta = -np.exp(-q * T) * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Use 'call' or 'put'.")

    gamma = np.exp(-q * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))
    vega = S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

    return option_price, delta, gamma, vega

In [5]:
# question 1.4
def calculate_option_price(S0, K, T, r, q, sigma, delta_s, num_simulations):
    np.random.seed(42)
    z = BoxMuller(num_simulations)
    # stock price
    ST = S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_plus = (S0 + delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    ST_minus = (S0 - delta_s) * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * z)
    # option price
    option_payoff = np.exp(-r * T) * (ST - K)
    option_payoff[option_payoff < 0] = 0
    option_price = np.mean(option_payoff)
    option_payoff_plus = np.exp(-r * T) * (ST_plus - K)
    option_payoff_plus[option_payoff_plus < 0] = 0
    option_price_plus = np.mean(option_payoff_plus)
    option_payoff_minus = np.exp(-r * T) * (ST_minus - K)
    option_payoff_minus[option_payoff_minus < 0] = 0
    option_price_minus = np.mean(option_payoff_minus)
    return option_price, option_price_plus, option_price_minus

# parameters
S0 = 41  # Current stock price
K = 42  # Strike price
T = 0.75  # Time to expiration (in years)
r = 0.03  # Risk-free interest rate
sigma = 0.25  # Volatility
q = 0.01  # Dividend yield

# bs value and delta
bsm_value, bsm_delta, bsm_gamma, bsm_vega = black_scholes(S0, K, T, r, q, sigma, "call")
print("BSM value:")
print(bsm_value, bsm_delta, bsm_gamma)

delta_s = 0.1 # delta_s

result_df = pd.DataFrame(columns=['Delta', 'Delta_diff', 'Gamma', 'Gamma_diff'])
for k in range(10):
    N = 10000 * (2**k)
    # option price
    option_price, option_price_plus, option_price_minus = calculate_option_price(S0, K, T, r, q, sigma, delta_s, N)
    # delta calculation
    delta = (option_price_plus - option_price_minus) / (2 * delta_s)
    # gamma calculation
    gamma = (option_price_plus - 2 * option_price + option_price_minus) / (delta_s)**2
    # diff from bsm value
    delta_diff = N**0.5 * abs(delta - bsm_delta)
    gamma_diff = N**0.5 * abs(gamma - bsm_gamma)
    # add to result_df
    result_df = result_df.append({'Delta': delta, 'Delta_diff': delta_diff, 'Gamma': gamma, 'Gamma_diff': gamma_diff}, ignore_index=True)

print("Monte Carlo value:")
print(result_df)
result_df.T.to_csv("./hw1_q1.csv")

BSM value:
3.3411791913217144 0.5224707292291643 0.04450878768741225
Monte Carlo value:
      Delta  Delta_diff     Gamma  Gamma_diff
0  0.518502    0.396897  0.055625    1.111604
1  0.526847    0.618956  0.049415    0.693819
2  0.518316    0.830858  0.042447    0.412379
3  0.521759    0.201419  0.041464    0.861245
4  0.523116    0.258202  0.045174    0.266100
5  0.522667    0.111172  0.043371    0.643798
6  0.522071    0.319613  0.042416    1.674472
7  0.521560    1.030298  0.043862    0.732198
8  0.522815    0.551551  0.045020    0.817886
9  0.522362    0.245126  0.044621    0.253167
